# CODIV-19

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.integrate import odeint
import torch
import pyro
import pyro.distributions as dist
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO
import pyro.poutine as poutine
from pyro.distributions import constraints
from datetime import timedelta

c:\Users\mated\anaconda3\envs\torch_win39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# def sir_step(S, I, R, beta, gamma, N):
#     dS = -beta * S * I / N
#     dI = beta * S * I / N - gamma * I
#     dR = gamma * I
#     next_S = S + dS
#     next_I = I + dI
#     next_R = R + dR
#     return next_S, next_I, next_R

In [3]:
def normalize_data(sir_data, population):
    """Normaliza los datos dividiendo por la población total"""
    sir_data['susceptible_norm'] = sir_data['susceptible'] / population
    sir_data['positive_norm'] = sir_data['positive'] / population
    sir_data['recovered_norm'] = sir_data['recovered'] / population
    return sir_data

In [4]:
# Define the SIR differential equations
def sir_model(y, t, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I
    dIdt = beta * S * I - gamma * I
    dRdt = gamma * I
    return [dSdt, dIdt, dRdt]

In [5]:
# Integrate the SIR equations over the time grid using given parameters
def sir_solution(y0, t, beta, gamma):
    # Ensure tensors are detached from the current graph
    t_np = t.detach().numpy()
    y0_np = [y.detach().numpy() for y in y0]
    beta_np = beta.detach().numpy()
    gamma_np = gamma.detach().numpy()
    return odeint(sir_model, y0_np, t_np, args=(beta_np, gamma_np))

In [6]:
def model(susceptible, infected, recovered):
    # Use wider bounds for priors
    beta = pyro.sample("beta", dist.Uniform(0.05, 0.5))
    gamma = pyro.sample("gamma", dist.Uniform(0.05, 0.5))
    
    # Initial conditions
    S0, I0, R0 = susceptible[0], infected[0], recovered[0]
    N = S0 + I0 + R0
    
    # Predictions
    S, I, R = S0, I0, R0
    S_pred = [S0]
    I_pred = [I0]
    R_pred = [R0]
    
    for _ in range(1, len(susceptible)):
        dS = -beta * S * I / N
        dI = beta * S * I / N - gamma * I
        dR = gamma * I
        
        S = torch.clamp(S + dS, min=0.0)
        I = torch.clamp(I + dI, min=0.0)
        R = torch.clamp(R + dR, min=0.0)
        
        S_pred.append(S)
        I_pred.append(I)
        R_pred.append(R)
    
    S_pred = torch.stack(S_pred)
    I_pred = torch.stack(I_pred)
    R_pred = torch.stack(R_pred)
    
    # Observations with larger noise scale
    with pyro.plate("data", len(susceptible)):
        pyro.sample("obs_S", dist.Normal(S_pred, 1000.0), obs=susceptible)
        pyro.sample("obs_I", dist.Normal(I_pred, 1000.0), obs=infected)
        pyro.sample("obs_R", dist.Normal(R_pred, 1000.0), obs=recovered)

In [7]:
# Load the data
us_daily_data = pd.read_csv("data/covid/us_covid19_daily.csv")

In [8]:
# Convert date columns to datetime format
us_daily_data['date'] = pd.to_datetime(us_daily_data['date'], format="%Y%m%d")

In [9]:
# Extract relevant columns for SIR modeling
sir_data = us_daily_data[['date', 'positive', 'death']].sort_values(by='date').reset_index(drop=True)

In [10]:
# Calculate the number of recovered individuals
recovery_delay = 14
sir_data['recovered'] = sir_data['positive'].shift(recovery_delay) - sir_data['death'].shift(recovery_delay)
sir_data['recovered'].fillna(0, inplace=True)

C:\Users\mated\AppData\Local\Temp\ipykernel_17112\2596338286.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sir_data['recovered'].fillna(0, inplace=True)


In [11]:
# Fill with 0 the missing values of deaths
sir_data['death'].fillna(0, inplace=True)

C:\Users\mated\AppData\Local\Temp\ipykernel_17112\3317233652.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sir_data['death'].fillna(0, inplace=True)


In [12]:
# For simplicity, we'll consider the entire US population as susceptible initially
us_population = 328_200_000  # US population estimate
sir_data['susceptible'] = us_population - sir_data['positive'] - sir_data['recovered']

In [13]:
sir_data.describe()

,date,positive,death,recovered,susceptible
count,221,2.210000e+02,221.000000,2.210000e+02,2.210000e+02
mean,2020-05-11 00:00:00,1.789319e+06,70660.864253,1.371226e+06,3.250395e+08
min,2020-01-22 00:00:00,0.000000e+00,0.000000,-4.000000e+00,3.170940e+08
25%,2020-03-17 00:00:00,1.034200e+04,123.000000,1.120000e+02,3.231489e+08
50%,2020-05-11 00:00:00,1.352463e+06,77512.000000,9.405530e+05,3.259070e+08
75%,2020-07-05 00:00:00,2.890899e+06,122576.000000,2.160187e+06,3.281895e+08
max,2020-08-29 00:00:00,5.928381e+06,174768.000000,5.177592e+06,3.282000e+08
std,NaN,1.835068e+06,60821.850930,1.523607e+06,3.355310e+06


In [14]:
sir_data = normalize_data(sir_data, us_population)

In [15]:
# Convert data to PyTorch tensors
susceptible_t = torch.tensor(sir_data['susceptible_norm'].values, dtype=torch.float)
infected_t = torch.tensor(sir_data['positive_norm'].values, dtype=torch.float)
recovered_t = torch.tensor(sir_data['recovered_norm'].values, dtype=torch.float)

In [16]:
def guide(susceptible, infected, recovered):
    # Use sigmoid to ensure parameters stay in valid range
    beta_loc = pyro.param('beta_loc', 
                         torch.tensor(0.2),
                         constraint=dist.constraints.interval(0.05, 0.5))
    gamma_loc = pyro.param('gamma_loc',
                          torch.tensor(0.2),
                          constraint=dist.constraints.interval(0.05, 0.5))
    
    # Sample parameters
    beta = pyro.sample('beta', dist.Delta(beta_loc))
    gamma = pyro.sample('gamma', dist.Delta(gamma_loc))

In [17]:
def plot_sir_results(sir_data, beta, gamma, susceptible_t, infected_t, recovered_t):
    """
    Plot the actual vs predicted SIR curves using the estimated parameters
    """
    # Generate predictions
    t = np.arange(len(sir_data))
    initial_conditions = [
        susceptible_t[0].item(),
        infected_t[0].item(),
        recovered_t[0].item()
    ]
    
    solution = odeint(
        sir_model,
        initial_conditions,
        t,
        args=(beta, gamma)
    )
    
    # Create the plot
    plt.figure(figsize=(15, 10))
    
    # Plot actual data
    plt.plot(sir_data['date'], susceptible_t.numpy(), 'b.', alpha=0.5, label='Actual Susceptible')
    plt.plot(sir_data['date'], infected_t.numpy(), 'r.', alpha=0.5, label='Actual Infected')
    plt.plot(sir_data['date'], recovered_t.numpy(), 'g.', alpha=0.5, label='Actual Recovered')
    
    # Plot predictions
    plt.plot(sir_data['date'], solution[:, 0], 'b-', label='Predicted Susceptible')
    plt.plot(sir_data['date'], solution[:, 1], 'r-', label='Predicted Infected')
    plt.plot(sir_data['date'], solution[:, 2], 'g-', label='Predicted Recovered')
    
    plt.title('SIR Model: Actual vs Predicted')
    plt.xlabel('Date')
    plt.ylabel('Population')
    plt.legend()
    plt.xticks(rotation=45)
    plt.grid(True)
    
    return solution

In [18]:
def calculate_metrics(actual_data, predicted_data):
    """
    Calculate error metrics for model evaluation
    """
    mse = np.mean((actual_data - predicted_data) ** 2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(actual_data - predicted_data))
    
    return {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae
    }

def make_future_predictions(beta, gamma, last_values, days_to_predict):
    """
    Make predictions for future dates using the fitted model
    """
    t_future = np.arange(days_to_predict)
    future_solution = odeint(
        sir_model,
        last_values,
        t_future,
        args=(beta, gamma)
    )
    
    return future_solution

In [19]:
def plot_loss_curve(losses):
    """
    Plot the training loss curve
    """
    plt.figure(figsize=(10, 6))
    plt.plot(losses)
    plt.title('Training Loss over Iterations')
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.grid(True)

In [20]:
def analyze_results(sir_data, beta_estimated, gamma_estimated, susceptible_t, infected_t, recovered_t, losses):
    """
    Comprehensive analysis of the SIR model results
    """
    # Plot the SIR curves
    solution = plot_sir_results(sir_data, beta_estimated, gamma_estimated, susceptible_t, infected_t, recovered_t)
    plt.show()
    
    # Plot loss curve
    plot_loss_curve(losses)
    plt.show()
    
    # Calculate metrics for each compartment
    metrics = {
        'Susceptible': calculate_metrics(susceptible_t.numpy(), solution[:, 0]),
        'Infected': calculate_metrics(infected_t.numpy(), solution[:, 1]),
        'Recovered': calculate_metrics(recovered_t.numpy(), solution[:, 2])
    }
    
    # Print metrics
    print("\nModel Performance Metrics:")
    for compartment, metric_values in metrics.items():
        print(f"\n{compartment}:")
        for metric_name, value in metric_values.items():
            print(f"{metric_name}: {value:,.2f}")
    
    # Calculate R0
    r0 = beta_estimated / gamma_estimated
    print(f"\nBasic Reproduction Number (R0): {r0:.2f}")
    
    # Make future predictions (30 days)
    days_to_predict = 30
    last_values = [
        susceptible_t[-1].item(),
        infected_t[-1].item(),
        recovered_t[-1].item()
    ]
    
    future_predictions = make_future_predictions(
        beta_estimated,
        gamma_estimated,
        last_values,
        days_to_predict
    )
    
    # Plot future predictions
    future_dates = [sir_data['date'].iloc[-1] + timedelta(days=x) for x in range(1, days_to_predict + 1)]
    
    plt.figure(figsize=(15, 10))
    plt.plot(future_dates, future_predictions[:, 0], 'b-', label='Predicted Susceptible')
    plt.plot(future_dates, future_predictions[:, 1], 'r-', label='Predicted Infected')
    plt.plot(future_dates, future_predictions[:, 2], 'g-', label='Predicted Recovered')
    plt.title('30-Day SIR Predictions')
    plt.xlabel('Date')
    plt.ylabel('Population')
    plt.legend()
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.show()
    
    return metrics, future_predictions

In [21]:
# Data conversion
susceptible_t = torch.tensor(susceptible_t).float()
infected_t = torch.tensor(infected_t).float()
recovered_t = torch.tensor(recovered_t).float()

C:\Users\mated\AppData\Local\Temp\ipykernel_17112\1653134527.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  susceptible_t = torch.tensor(susceptible_t).float()
C:\Users\mated\AppData\Local\Temp\ipykernel_17112\1653134527.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  infected_t = torch.tensor(infected_t).float()
C:\Users\mated\AppData\Local\Temp\ipykernel_17112\1653134527.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  recovered_t = torch.tensor(recovered_t).float()


In [22]:
def sir_model(y, t, beta, gamma):
    """Modelo SIR modificado con restricciones de no negatividad"""
    S, I, R = y
    # Asegurar que las proporciones estén entre 0 y 1
    S = np.clip(S, 0, 1)
    I = np.clip(I, 0, 1)
    R = np.clip(R, 0, 1)
    
    # Normalizar para mantener S + I + R = 1
    total = S + I + R
    S, I, R = S/total, I/total, R/total
    
    dSdt = -beta * S * I
    dIdt = beta * S * I - gamma * I
    dRdt = gamma * I
    
    return [dSdt, dIdt, dRdt]

def model(susceptible, infected, recovered):
    """Modelo probabilístico mejorado"""
    # Priors más restrictivos basados en literatura del COVID-19
    beta = pyro.sample("beta", dist.Uniform(0.2, 0.4))  # R0 típico entre 2-4
    gamma = pyro.sample("gamma", dist.Uniform(0.1, 0.2))  # Período de recuperación ~5-10 días
    
    # Normalizar condiciones iniciales
    total = susceptible[0] + infected[0] + recovered[0]
    S0 = susceptible[0] / total
    I0 = infected[0] / total
    R0 = recovered[0] / total
    
    # Predicciones
    S, I, R = S0, I0, R0
    S_pred = [S0]
    I_pred = [I0]
    R_pred = [R0]
    
    for _ in range(1, len(susceptible)):
        # Calcular cambios
        dS = -beta * S * I
        dI = beta * S * I - gamma * I
        dR = gamma * I
        
        # Actualizar estados con restricciones
        S = torch.clamp(S + dS, min=0.0, max=1.0)
        I = torch.clamp(I + dI, min=0.0, max=1.0)
        R = torch.clamp(R + dR, min=0.0, max=1.0)
        
        # Normalizar para mantener suma = 1
        total = S + I + R
        S = S / total
        I = I / total
        R = R / total
        
        S_pred.append(S)
        I_pred.append(I)
        R_pred.append(R)
    
    S_pred = torch.stack(S_pred)
    I_pred = torch.stack(I_pred)
    R_pred = torch.stack(R_pred)
    
    # Observaciones con escala de ruido adaptativa
    noise_scale = 0.01  # 1% de error
    with pyro.plate("data", len(susceptible)):
        pyro.sample("obs_S", dist.Normal(S_pred, noise_scale), obs=susceptible)
        pyro.sample("obs_I", dist.Normal(I_pred, noise_scale), obs=infected)
        pyro.sample("obs_R", dist.Normal(R_pred, noise_scale), obs=recovered)

def guide(susceptible, infected, recovered):
    """Guía mejorada con restricciones más estrictas"""
    beta_loc = pyro.param('beta_loc', 
                         torch.tensor(0.3),
                         constraint=dist.constraints.interval(0.2, 0.4))
    gamma_loc = pyro.param('gamma_loc',
                          torch.tensor(0.15),
                          constraint=dist.constraints.interval(0.1, 0.2))
    
    beta = pyro.sample('beta', dist.Delta(beta_loc))
    gamma = pyro.sample('gamma', dist.Delta(gamma_loc))

In [23]:
def plot_results(sir_data, beta, gamma, susceptible_t, infected_t, recovered_t):
    """Función mejorada para visualizar resultados"""
    # Normalizar datos
    total = susceptible_t + infected_t + recovered_t
    susceptible_norm = susceptible_t / total
    infected_norm = infected_t / total
    recovered_norm = recovered_t / total
    
    # Generar predicciones
    t = np.arange(len(sir_data))
    initial_conditions = [
        susceptible_norm[0].item(),
        infected_norm[0].item(),
        recovered_norm[0].item()
    ]
    
    solution = odeint(
        sir_model,
        initial_conditions,
        t,
        args=(beta, gamma)
    )
    
    plt.figure(figsize=(15, 10))
    plt.plot(sir_data['date'], susceptible_norm, 'b.', alpha=0.5, label='Actual Susceptible')
    plt.plot(sir_data['date'], infected_norm, 'r.', alpha=0.5, label='Actual Infected')
    plt.plot(sir_data['date'], recovered_norm, 'g.', alpha=0.5, label='Actual Recovered')
    
    plt.plot(sir_data['date'], solution[:, 0], 'b-', label='Predicted Susceptible')
    plt.plot(sir_data['date'], solution[:, 1], 'r-', label='Predicted Infected')
    plt.plot(sir_data['date'], solution[:, 2], 'g-', label='Predicted Recovered')
    
    plt.title('Normalized SIR Model: Actual vs Predicted')
    plt.xlabel('Date')
    plt.ylabel('Population Proportion')
    plt.legend()
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.ylim(0, 1)
    
    return solution

In [24]:
import numpy as np
import pandas as pd
import torch
import pyro
import pyro.distributions as dist
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO
import matplotlib.pyplot as plt
from scipy.integrate import odeint

def prepare_seirhd_data(data, population):
    """
    Prepara los datos para el modelo SEIRHD, calculando primero los expuestos
    """
    # Crear una copia para no modificar los datos originales
    data = data.copy()
    
    # Calcular nuevos casos si no existe la columna
    if 'new_cases' not in data.columns:
        data['new_cases'] = data['positive'].diff().fillna(0)
    
    # Calcular expuestos usando un período de incubación de 5 días
    # Los expuestos de hoy son los nuevos casos de 5 días después
    data['exposed'] = data['new_cases'].shift(-5).fillna(0)
    
    # Calcular susceptibles
    data['susceptible'] = population - data['positive'] - data['recovered'] - data['death']
    
    # Usar hospitalizedCurrently para H, rellenando valores nulos
    data['hospitalized_current'] = data['hospitalizedCurrently'].fillna(method='ffill').fillna(0)
    
    # Calcular tasas por población
    data['susceptible_norm'] = data['susceptible'] / population
    data['exposed_norm'] = data['exposed'] / population
    data['infected_norm'] = (data['positive'] - data['hospitalized_current']) / population
    data['hospitalized_norm'] = data['hospitalized_current'] / population
    data['recovered_norm'] = data['recovered'] / population
    data['death_norm'] = data['death'] / population
    
    # Asegurar que no hay valores negativos
    columns_norm = ['susceptible_norm', 'exposed_norm', 'infected_norm', 
                   'hospitalized_norm', 'recovered_norm', 'death_norm']
    for col in columns_norm:
        data[col] = data[col].clip(lower=0)
    
    # Normalizar para que la suma sea 1
    total = data[columns_norm].sum(axis=1)
    for col in columns_norm:
        data[col] = data[col] / total
    
    # Eliminar filas con valores NaN
    data = data.dropna(subset=columns_norm)
    
    return data

def calculate_effective_reproduction_number(data, beta, gamma_i, gamma_h):
    """
    Calcula el número de reproducción efectivo (Rt) a lo largo del tiempo
    """
    # Rt = beta * S(t) / (gamma_i + gamma_h)
    Rt = beta * data['susceptible_norm'] / (gamma_i + gamma_h)
    return Rt

def print_model_statistics(data, params):
    """
    Imprime estadísticas importantes del modelo
    """
    print("\nParámetros estimados del modelo:")
    for param_name, value in params.items():
        print(f"{param_name}: {value:.4f}")
    
    print("\nEstadísticas del modelo:")
    print(f"Tasa de hospitalización promedio: {(data['hospitalized_norm'] / data['infected_norm']).mean():.2%}")
    print(f"Tasa de letalidad (CFR): {(data['death_norm'].iloc[-1] / data['positive'].iloc[-1]):.2%}")
    print(f"Tasa de recuperación: {(data['recovered_norm'].iloc[-1] / data['positive'].iloc[-1]):.2%}")
    
def seirhd_model(state, t, beta, sigma, gamma_i, gamma_h, eta, mu_i, mu_h):
    """
    SEIRHD Model differential equations
    
    Parameters:
    - beta: tasa de infección
    - sigma: tasa de progresión de expuesto a infectado (1/período de incubación)
    - gamma_i: tasa de recuperación de infectados
    - gamma_h: tasa de recuperación de hospitalizados
    - eta: tasa de hospitalización
    - mu_i: tasa de mortalidad de infectados
    - mu_h: tasa de mortalidad de hospitalizados
    
    States:
    S: Susceptible
    E: Exposed (expuestos)
    I: Infected (infectados no hospitalizados)
    H: Hospitalized
    R: Recovered
    D: Deceased
    """
    S, E, I, H, R, D = state
    
    # Asegurar que las proporciones están entre 0 y 1
    S = np.clip(S, 0, 1)
    E = np.clip(E, 0, 1)
    I = np.clip(I, 0, 1)
    H = np.clip(H, 0, 1)
    R = np.clip(R, 0, 1)
    D = np.clip(D, 0, 1)
    
    # Normalizar para mantener suma = 1
    total = S + E + I + H + R + D
    S, E, I, H, R, D = S/total, E/total, I/total, H/total, R/total, D/total
    
    # Ecuaciones del modelo SEIRHD
    dSdt = -beta * S * (I + H)
    dEdt = beta * S * (I + H) - sigma * E
    dIdt = sigma * E - (gamma_i + eta + mu_i) * I
    dHdt = eta * I - (gamma_h + mu_h) * H
    dRdt = gamma_i * I + gamma_h * H
    dDdt = mu_i * I + mu_h * H
    
    return [dSdt, dEdt, dIdt, dHdt, dRdt, dDdt]

def seirhd_model_pyro(susceptible, exposed, infected, hospitalized, recovered, deceased):
    """
    Implementación probabilística del modelo SEIRHD en Pyro
    """
    # Priors informativas basadas en literatura de COVID-19
    beta = pyro.sample("beta", dist.Uniform(0.2, 0.4))
    sigma = pyro.sample("sigma", dist.Uniform(0.15, 0.25))  # 4-7 días de incubación
    gamma_i = pyro.sample("gamma_i", dist.Uniform(0.08, 0.15))  # 7-12 días recuperación sin hospital
    gamma_h = pyro.sample("gamma_h", dist.Uniform(0.05, 0.1))   # 10-20 días recuperación en hospital
    eta = pyro.sample("eta", dist.Uniform(0.02, 0.05))         # 2-5% tasa hospitalización
    mu_i = pyro.sample("mu_i", dist.Uniform(0.001, 0.005))     # 0.1-0.5% mortalidad sin hospital
    mu_h = pyro.sample("mu_h", dist.Uniform(0.01, 0.03))       # 1-3% mortalidad en hospital
    
    # Condiciones iniciales
    S0, E0, I0, H0, R0, D0 = (
        susceptible[0], exposed[0], infected[0],
        hospitalized[0], recovered[0], deceased[0]
    )
    
    # Predicciones
    S, E, I, H, R, D = S0, E0, I0, H0, R0, D0
    S_pred = [S0]
    E_pred = [E0]
    I_pred = [I0]
    H_pred = [H0]
    R_pred = [R0]
    D_pred = [D0]
    
    for _ in range(1, len(susceptible)):
        # Calcular cambios
        dS = -beta * S * (I + H)
        dE = beta * S * (I + H) - sigma * E
        dI = sigma * E - (gamma_i + eta + mu_i) * I
        dH = eta * I - (gamma_h + mu_h) * H
        dR = gamma_i * I + gamma_h * H
        dD = mu_i * I + mu_h * H
        
        # Actualizar estados con restricciones
        S = torch.clamp(S + dS, min=0.0, max=1.0)
        E = torch.clamp(E + dE, min=0.0, max=1.0)
        I = torch.clamp(I + dI, min=0.0, max=1.0)
        H = torch.clamp(H + dH, min=0.0, max=1.0)
        R = torch.clamp(R + dR, min=0.0, max=1.0)
        D = torch.clamp(D + dD, min=0.0, max=1.0)
        
        # Normalizar
        total = S + E + I + H + R + D
        S, E, I, H, R, D = S/total, E/total, I/total, H/total, R/total, D/total
        
        # Almacenar predicciones
        S_pred.append(S)
        E_pred.append(E)
        I_pred.append(I)
        H_pred.append(H)
        R_pred.append(R)
        D_pred.append(D)
    
    S_pred = torch.stack(S_pred)
    E_pred = torch.stack(E_pred)
    I_pred = torch.stack(I_pred)
    H_pred = torch.stack(H_pred)
    R_pred = torch.stack(R_pred)
    D_pred = torch.stack(D_pred)
    
    # Observaciones con ruido adaptativo
    noise_scale = 0.01
    with pyro.plate("data", len(susceptible)):
        pyro.sample("obs_S", dist.Normal(S_pred, noise_scale), obs=susceptible)
        pyro.sample("obs_E", dist.Normal(E_pred, noise_scale), obs=exposed)
        pyro.sample("obs_I", dist.Normal(I_pred, noise_scale), obs=infected)
        pyro.sample("obs_H", dist.Normal(H_pred, noise_scale), obs=hospitalized)
        pyro.sample("obs_R", dist.Normal(R_pred, noise_scale), obs=recovered)
        pyro.sample("obs_D", dist.Normal(D_pred, noise_scale), obs=deceased)

def guide_seirhd(susceptible, exposed, infected, hospitalized, recovered, deceased):
    """
    Función guía para el modelo SEIRHD que define la distribución variacional
    """
    # Parámetros para beta (tasa de infección)
    beta_loc = pyro.param('beta_loc', 
                         torch.tensor(0.3),
                         constraint=dist.constraints.interval(0.2, 0.4))
    
    # Parámetros para sigma (tasa de progresión de expuesto a infectado)
    sigma_loc = pyro.param('sigma_loc',
                          torch.tensor(0.2),
                          constraint=dist.constraints.interval(0.15, 0.25))
    
    # Parámetros para gamma_i (tasa de recuperación de infectados)
    gamma_i_loc = pyro.param('gamma_i_loc',
                            torch.tensor(0.1),
                            constraint=dist.constraints.interval(0.08, 0.15))
    
    # Parámetros para gamma_h (tasa de recuperación de hospitalizados)
    gamma_h_loc = pyro.param('gamma_h_loc',
                            torch.tensor(0.07),
                            constraint=dist.constraints.interval(0.05, 0.1))
    
    # Parámetros para eta (tasa de hospitalización)
    eta_loc = pyro.param('eta_loc',
                        torch.tensor(0.03),
                        constraint=dist.constraints.interval(0.02, 0.05))
    
    # Parámetros para mu_i (tasa de mortalidad de infectados)
    mu_i_loc = pyro.param('mu_i_loc',
                         torch.tensor(0.002),
                         constraint=dist.constraints.interval(0.001, 0.005))
    
    # Parámetros para mu_h (tasa de mortalidad de hospitalizados)
    mu_h_loc = pyro.param('mu_h_loc',
                         torch.tensor(0.02),
                         constraint=dist.constraints.interval(0.01, 0.03))
    
    # Muestrear parámetros
    beta = pyro.sample('beta', dist.Delta(beta_loc))
    sigma = pyro.sample('sigma', dist.Delta(sigma_loc))
    gamma_i = pyro.sample('gamma_i', dist.Delta(gamma_i_loc))
    gamma_h = pyro.sample('gamma_h', dist.Delta(gamma_h_loc))
    eta = pyro.sample('eta', dist.Delta(eta_loc))
    mu_i = pyro.sample('mu_i', dist.Delta(mu_i_loc))
    mu_h = pyro.sample('mu_h', dist.Delta(mu_h_loc))
    
def plot_seirhd_results(data, solution, title="SEIRHD Model Results"):
    """
    Visualizar resultados del modelo SEIRHD
    """
    plt.figure(figsize=(15, 10))
    
    # Plot datos reales
    plt.plot(data.index, data['susceptible_norm'], 'b.', alpha=0.5, label='Actual Susceptible')
    plt.plot(data.index, data['exposed_norm'], 'y.', alpha=0.5, label='Actual Exposed')
    plt.plot(data.index, data['infected_norm'], 'r.', alpha=0.5, label='Actual Infected')
    plt.plot(data.index, data['hospitalized_norm'], 'm.', alpha=0.5, label='Actual Hospitalized')
    plt.plot(data.index, data['recovered_norm'], 'g.', alpha=0.5, label='Actual Recovered')
    plt.plot(data.index, data['death_norm'], 'k.', alpha=0.5, label='Actual Deceased')
    
    # Plot predicciones
    plt.plot(data.index, solution[:, 0], 'b-', label='Predicted Susceptible')
    plt.plot(data.index, solution[:, 1], 'y-', label='Predicted Exposed')
    plt.plot(data.index, solution[:, 2], 'r-', label='Predicted Infected')
    plt.plot(data.index, solution[:, 3], 'm-', label='Predicted Hospitalized')
    plt.plot(data.index, solution[:, 4], 'g-', label='Predicted Recovered')
    plt.plot(data.index, solution[:, 5], 'k-', label='Predicted Deceased')
    
    plt.title(title)
    plt.xlabel('Date')
    plt.ylabel('Population Proportion')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)
    plt.ylim(0, 1)
    
    return plt

In [25]:
# Para nuevo metodo
processed_data = prepare_seirhd_data(us_daily_data, us_population)

# Convertir a tensores
susceptible_t = torch.tensor(processed_data['susceptible_norm'].values, dtype=torch.float)
exposed_t = torch.tensor(processed_data['exposed_norm'].values, dtype=torch.float)
infected_t = torch.tensor(processed_data['infected_norm'].values, dtype=torch.float)
hospitalized_t = torch.tensor(processed_data['hospitalized_norm'].values, dtype=torch.float)
recovered_t = torch.tensor(processed_data['recovered_norm'].values, dtype=torch.float)
deceased_t = torch.tensor(processed_data['death_norm'].values, dtype=torch.float)


C:\Users\mated\AppData\Local\Temp\ipykernel_17112\3236758816.py:30: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['hospitalized_current'] = data['hospitalizedCurrently'].fillna(method='ffill').fillna(0)


In [26]:
# Configurar y entrenar el modelo
pyro.clear_param_store()
optimizer = Adam({"lr": 0.001})
svi = SVI(seirhd_model_pyro, guide_seirhd, optimizer, loss=Trace_ELBO())

In [27]:
# # Setup for SVI
# pyro.clear_param_store()
# optimizer = Adam({"lr": 0.01})
# svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

In [28]:
# # # Train the model
# # num_iterations = 5000
# # for j in range(num_iterations):
# #     loss = svi.step(susceptible_t, infected_t, recovered_t)
# #     if j % 500 == 0:
# #         print("[iteration %04d] loss: %.4f" % (j + 1, loss / len(susceptible_t)))
# # Running SVI
# losses = []
# num_iterations = 300
# for j in range(num_iterations):
#     loss = svi.step(susceptible_t, infected_t, recovered_t)
#     losses.append(loss)
#     if j % 100 == 0:
#         print("[iteration %04d] loss: %.4f" % (j + 1, loss / len(susceptible_t)))

# losses[-1]

In [29]:
# Entrenar
num_iterations = 1000
losses = []
for j in range(num_iterations):
    loss = svi.step(susceptible_t, exposed_t, infected_t, 
                    hospitalized_t, recovered_t, deceased_t)
    losses.append(loss)
    if j % 100 == 0:
        print(f"[iteration {j+1:04d}] loss: {loss:.4f}")

[iteration 0001] loss: 880204.1799
[iteration 0101] loss: 856300.1463
[iteration 0201] loss: 830038.8307
[iteration 0301] loss: 801403.4916
[iteration 0401] loss: 770513.2543
[iteration 0501] loss: 737620.1463
[iteration 0601] loss: 703111.7174
[iteration 0701] loss: 667492.1094
[iteration 0801] loss: 631345.3261
[iteration 0901] loss: 595282.3060


In [30]:
# # Extract the inferred parameters
# beta_estimated = pyro.param("beta_loc").item()
# gamma_estimated = pyro.param("gamma_loc").item()

# print(f"Estimated beta: {beta_estimated}")
# print(f"Estimated gamma: {gamma_estimated}")

In [ ]:
susceptible_t

In [31]:
# Obtener parámetros
beta = pyro.param("beta_loc").item()
sigma = pyro.param("sigma_loc").item()
gamma = pyro.param("gamma_loc").item()
mu = pyro.param("mu_loc").item()

KeyError: 'gamma_loc'

In [ ]:
# Visualizar resultados
solution = plot_results(sir_data, beta_estimated, gamma_estimated, 
                       susceptible_t, infected_t, recovered_t)

In [ ]:
metrics, future_predictions = analyze_results(
    sir_data,
    beta_estimated,
    gamma_estimated,
    susceptible_t,
    infected_t,
    recovered_t,
    losses
)


In [ ]:
# Generar fechas futuras para las predicciones
future_dates = [sir_data['date'].iloc[-1] + timedelta(days=x) for x in range(1, 31)]